# Import thư viện cần thiết

In [ ]:
import re #regex
import csv
import os
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from pandas import DataFrame
import urllib.request
import joblib #load, dump pkl
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from underthesea import word_tokenize #word_tokenize
import numpy as np
import matplotlib.pyplot as plt
import transformers as ppb # load model BERT
from transformers import BertModel, BertTokenizer
import torch
from bs4 import BeautifulSoup

# Prediction

## Lấy dữ liệu comment từ  URL

In [ ]:
def getAllComment(url):
    browser = webdriver.Chrome()
    browser.maximize_window()

    records = []
    browser.get(url)
    # tat quang cao
    browser.find_element_by_css_selector('html').send_keys(Keys.ESCAPE)

    # scroll
    time.sleep(1)
    total_height = int(browser.execute_script(
        "return document.body.scrollHeight"))
    for i in range(1, total_height, 5):
        browser.execute_script("window.scrollTo(0, {});".format(i))
    time.sleep(3)

    html_source = browser.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    button = browser.find_elements_by_css_selector(
        'button.align-right.secondary.slidedown-button')

    if(len(button) > 0):
        button[0].click()

    html_source = browser.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    commentDivs = soup.findAll(
        "div", {"class": "style__StyledComment-sc-103p4dk-5 dDtAUu review-comment"})
    for commentDiv in commentDivs:
        cmt = commentDiv.find(
            "div", {"class": "review-comment__content"}).text
        if(len(cmt) > 0):
            records.append(cmt)

    navigationButton = browser.find_elements_by_css_selector(
        'a.btn.next')

    while len(navigationButton) > 0:
        navigationButton[0].click()
        time.sleep(1)
        html_source = browser.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        commentDivs = soup.findAll(
            "div", {"class": "style__StyledComment-sc-103p4dk-5 dDtAUu review-comment"})
        for commentDiv in commentDivs:
            cmt = commentDiv.find(
                "div", {"class": "review-comment__content"}).text
            if(len(cmt) > 0):
                records.append(cmt)
        navigationButton = browser.find_elements_by_css_selector(
            'a.btn.next')
    browser.quit()

    return {'comment':records}

## Xử lý dữ liệu

### Word tokenizer 

Tách các comment thành từ hoặc cụm từ có nghĩa, fomart="text" sẽ cho ra kết quả các từ đơn và từ ghép có nghĩa.

In [ ]:
def tokenizer(row):
    return word_tokenize(row, format="text")

### Chuẩn hoá comment
Dữ liệu hiện đang chứa những kí tự như dấu chấm câu, kí tự đặc biệt, icon,... Những kí tự này không cần thiết cho việc khai phá dữ liệu nên ta sẽ tiến hành loại bỏ chúng. Các bước loại bỏ:
- Xoá đi các dấu chấm câu như (. , ? ; ...)
- Xoá đi các kí tự đặc biệt: @,#,*,...
- Xoá khoảng trắng đầu và cuối mỗi comment, xoá  khoảng trắng dư thừa giữa các từ.
- Xoá đi icons và emoji.

In [ ]:
def standardize_data(row):
    row = re.sub(r"[\.,\?]+$-", "", row)
    row = re.sub(r'[^\w]',' ', row) 
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ")
    row = row.strip()
    row = " ".join(row.split())
    return row

### Hàm xử lý dữ liệu

Hàm chuẩn hoá sẽ thực hiện các công việc sau:
- Chuẩn hoá comment dùng hàm standardize_data ở trên.
- Loại bỏ các comment trùng lặp.
- Tokenizer các comment.
- Trả về Series comment.

In [ ]:
def processing_data(data):
    # 1. Standardize data
    data_frame = pd.DataFrame(data=data)
    data_frame.drop_duplicates(subset ="comment", keep = 'first', inplace = True) 
    data_frame['comment'] = data_frame['comment'].apply(standardize_data)
    # 2. Tokenizer
    data_frame['comment'] = data_frame['comment'].apply(tokenizer)
    # 3. Embedding
    X_val = data_frame['comment']
    print('comments')
    print(X_val)
    return X_val

## Load Pre-train model BERT

In [ ]:
def load_pretrainModel(data):
    '''
    Load pretrain model/ tokenizers
    Return : features
    '''
    model = BertModel.from_pretrained('bert-base-uncased')
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    #encode lines
    tokenized = data.apply((lambda x: tokenizer.encode(x, add_special_tokens = True,max_length=512,truncation=True)))

    # get lenght max of tokenized
    max_len = 0
    for i in tokenized.values:
        if len(i) > max_len:
            max_len = len(i)
    print('max len:', max_len)

    # if lenght of tokenized not equal max_len , so padding value 0
    padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
    print('padded:', padded[1])
    print('len padded:', padded.shape)

    #get attention mask ( 0: not has word, 1: has word)
    attention_mask = np.where(padded ==0, 0,1)
    print('attention mask:', attention_mask[1])

    # Convert input to tensor
    padded = torch.tensor(padded,dtype=torch.long)
    attention_mask = torch.tensor(attention_mask)


    # Load model
    with torch.no_grad():
        last_hidden_states = model(padded, attention_mask =attention_mask)
    #     print('last hidden states:', last_hidden_states)

    features = last_hidden_states[0][:,0,:].numpy()
    print('features:', features)
    
    return features


## Predict

- Hàm predict sẽ nhận vào url sản phẩm bất kì từ https://tiki.vn/. Tiến hành crawl các comment và chuẩn hoá dữ liệu. Sau đó đưa vào BERT model để rút trích features và đánh giá bằng model chúng ta đã train ở bước Training model.
- Hàm anlyze sẽ nhận vào kết quả predict và đếm số lượng các loại bình luận được gán nhãn. Nếu bình luận tích cực có số lượng lớn ·hơn sẽ gợi ý người dùng mua sản phẩm và ngược lại.

In [ ]:
def analyze(result):
    good = np.count_nonzero(result)
    bad = len(result) - good
    
    print("Số bình luận tích cực và trung lập: ", good)
    print("Số bình luận tiêu cực: ", bad)

    if good>bad:
        return "Mặt hàng tốt! Bạn có thể mua ngay."
    else:
        return "Có nhiều bình luận xấu! Cần xem xét kĩ trước khi quyết định mua."

def predict(url):
    # 1. Load URL and print comments
    data = getAllComment(url)
    data = processing_data(data)
    features = load_pretrainModel(data)
    # 2. Load weights
    model = joblib.load('save_model.pkl')
    # 3. Result
    result = model.predict(features)
    print(result)
    print(analyze(result))

Chạy thử nghiệm quá trình gợi ý mua sản phẩm

In [ ]:
url = 'https://tiki.vn/macbook-air-2017-mqd32-13-inch-hang-chinh-hang-p721995.html'
predict(url)